In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

In [2]:
BASE_MODEL = r"/shared/4/models/llama2/llama-2-70b-chat"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer = LlamaTokenizer.from_pretrained(r"/shared/4/models/llama2/")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# Encoding the input text
prompt = "The future of AI is"

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]

In [19]:
response = model.generate(
    input_ids=input_ids,
    max_new_tokens=128,  # Assuming 128 is the desired maximum length
    temperature=0.1,
    top_p=0.75,
    repetition_penalty=1.1,
    #pad_token_id=tokenizer.eos_token_id,  # Ensure padding with the EOS token
    return_dict_in_generate=True,
    output_scores=True
    )

In [24]:
generated_text = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
generated_text

'The future of AI is in the hands of the people\nThe future of artificial intelligence (AI) is in the hands of the people. That’s according to a new report from the World Economic Forum, which found that 85% of business leaders believe that AI will have a positive impact on society and the economy over the next decade. However, only 37% of respondents said they were confident about their ability to manage the risks associated with AI.\nThe report also found that there is a significant gap between what businesses are doing now and what they need to do in order to be successful in the future.'

### Zero-shot

In [74]:
prompt_format = """
### Instruction:
What is the stance of the tweet below with respect to 'Legalization of Abortion'? If we can infer from the tweet that the tweeter supports 'Legalization of Abortion', please label it as 'in-favor'. \
If we can infer from the tweet that the tweeter is against 'Legalization of Abortion', please label is as 'against'. \
If we can infer from the tweet that the tweeter has a neutral stance towards 'Legalization of Abortion', please label it as 'none'. \
If there is no clue in the tweet to reveal the stance of the tweeter towards 'Legalization of Abortion', please also label is as 'none'. \
Please use exactly one word from the following 3 categories to label it: 'in-favor', 'against', 'neutral-or-unclear'. 

### Tweet: 
i really don't understand how some people are pro-choice. a life is a life no matter if it's 2 weeks old or 20 years old. 

### Response:
"""

In [75]:
input_ids = tokenizer(prompt_format, return_tensors="pt")["input_ids"]

In [76]:
response = model.generate(
    input_ids=input_ids,
    max_new_tokens=16,  # Assuming 128 is the desired maximum length
    temperature=0,
    top_p=0.75,
    repetition_penalty=1.1,
    #pad_token_id=tokenizer.eos_token_id,  # Ensure padding with the EOS token
    return_dict_in_generate=True,
    output_scores=True
    )

In [77]:
generated_text = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
print(generated_text)


### Instruction:
What is the stance of the tweet below with respect to 'Legalization of Abortion'? If we can infer from the tweet that the tweeter supports 'Legalization of Abortion', please label it as 'in-favor'. If we can infer from the tweet that the tweeter is against 'Legalization of Abortion', please label is as 'against'. If we can infer from the tweet that the tweeter has a neutral stance towards 'Legalization of Abortion', please label it as 'none'. If there is no clue in the tweet to reveal the stance of the tweeter towards 'Legalization of Abortion', please also label is as 'none'. Please use exactly one word from the following 3 categories to label it: 'in-favor', 'against', 'neutral-or-unclear'. 

### Tweet: 
i really don't understand how some people are pro-choice. a life is a life no matter if it's 2 weeks old or 20 years old. 

### Response:
- 'in-favor'
- 'against'
-


### Few-shot

In [78]:
prompt_format = """
### Instruction:
What is the stance of the tweet below with respect to 'Legalization of Abortion'? If we can infer from the tweet that the tweeter supports 'Legalization of Abortion', please label it as 'in-favor'. \
If we can infer from the tweet that the tweeter is against 'Legalization of Abortion', please label is as 'against'. \
If we can infer from the tweet that the tweeter has a neutral stance towards 'Legalization of Abortion', please label it as 'none'. \
If there is no clue in the tweet to reveal the stance of the tweeter towards 'Legalization of Abortion', please also label is as 'none'. \
Please use exactly one word from the following 3 categories to label it: 'in-favor', 'against', 'neutral-or-unclear'. 

### Tweet: 
Everyone deserves the right to make decisions about their own bodies and futures.

### Response:
in-favor

### Tweet: 
Life begins at conception, and every life has inherent value and the right to be protected. Abortion undermines this principle by ending an innocent life. 

### Response:
against

### Tweet: 
I believe that abortion is a deeply personal and complex issue, so why do we care?

### Response:
none

### Tweet:
i really don't understand how some people are pro-choice. a life is a life no matter if it's 2 weeks old or 20 years old. 
"""

In [79]:
input_ids = tokenizer(prompt_format, return_tensors="pt")["input_ids"]

In [80]:
response = model.generate(
    input_ids=input_ids,
    max_new_tokens=16,  # Assuming 128 is the desired maximum length
    temperature=0,
    top_p=0.75,
    repetition_penalty=1.1,
    #pad_token_id=tokenizer.eos_token_id,  # Ensure padding with the EOS token
    return_dict_in_generate=True,
    output_scores=True
    )

In [81]:
generated_text = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
print(generated_text)


### Instruction:
What is the stance of the tweet below with respect to 'Legalization of Abortion'? If we can infer from the tweet that the tweeter supports 'Legalization of Abortion', please label it as 'in-favor'. If we can infer from the tweet that the tweeter is against 'Legalization of Abortion', please label is as 'against'. If we can infer from the tweet that the tweeter has a neutral stance towards 'Legalization of Abortion', please label it as 'none'. If there is no clue in the tweet to reveal the stance of the tweeter towards 'Legalization of Abortion', please also label is as 'none'. Please use exactly one word from the following 3 categories to label it: 'in-favor', 'against', 'neutral-or-unclear'. 

### Tweet: 
Everyone deserves the right to make decisions about their own bodies and futures.

### Response:
in-favor

### Tweet: 
Life begins at conception, and every life has inherent value and the right to be protected. Abortion undermines this principle by ending an innocen